# Task 3

In [1]:
import pandas as pd

# Load data
df = pd.read_csv('../data/MachineLearningRating_v3.txt', delimiter='|', encoding='utf-8')

# Claim status
df['HasClaim'] = df['TotalClaims'] > 0

# Claim severity (filter rows with claims)
df['ClaimSeverity'] = df['TotalClaims']
df.loc[df['TotalClaims'] == 0, 'ClaimSeverity'] = None  # Avoid zero division

# Margin
df['Margin'] = df['TotalPremium'] - df['TotalClaims']

C:\Users\Bereket\AppData\Local\Temp\ipykernel_6532\1447819143.py:4: DtypeWarning: Columns (32,37) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/MachineLearningRating_v3.txt', delimiter='|', encoding='utf-8')


### H₀ #1: Risk difference by Province

In [2]:
import scipy.stats as stats

contingency = pd.crosstab(df['Province'], df['HasClaim'])
chi2, p, _, _ = stats.chi2_contingency(contingency)
print("Chi² Test for Claim Frequency by Province:")
print(f"Chi² = {chi2:.2f}, p = {p:.4f}")

Chi² Test for Claim Frequency by Province:
Chi² = 104.19, p = 0.0000


In [3]:
province_claims = df[df['HasClaim']].groupby('Province')['ClaimSeverity'].apply(list)
anova_result = stats.f_oneway(*province_claims.dropna())
print("ANOVA for Claim Severity by Province:")
print(f"F = {anova_result.statistic:.2f}, p = {anova_result.pvalue:.4f}")

ANOVA for Claim Severity by Province:
F = 4.83, p = 0.0000


### H₀ #2: Risk difference by ZipCode

In [4]:
zip_sample = df[df['HasClaim']].groupby('PostalCode').filter(lambda x: len(x) > 10)
grouped_zip = zip_sample.groupby('PostalCode')['ClaimSeverity'].apply(list)

anova_zip = stats.f_oneway(*grouped_zip)
print("ANOVA for Claim Severity by ZipCode:")
print(f"F = {anova_zip.statistic:.2f}, p = {anova_zip.pvalue:.4f}")


ANOVA for Claim Severity by ZipCode:
F = 2.56, p = 0.0000


### H₀ #3: Margin difference by ZipCode

In [5]:
margin_zip = df.groupby('PostalCode')['Margin'].apply(list)
anova_margin = stats.f_oneway(*margin_zip)
print("ANOVA for Margin by ZipCode:")
print(f"F = {anova_margin.statistic:.2f}, p = {anova_margin.pvalue:.4f}")


ANOVA for Margin by ZipCode:
F = 0.87, p = 0.9977


### H₀ #4: Risk difference by Gender

In [6]:
contingency = pd.crosstab(df['Gender'], df['HasClaim'])
chi2_gender, p_gender, _, _ = stats.chi2_contingency(contingency)
print(f"Chi² for Gender vs Claim Frequency: Chi² = {chi2_gender:.2f}, p = {p_gender:.4f}")

Chi² for Gender vs Claim Frequency: Chi² = 7.26, p = 0.0266


In [7]:
male_claims = df[(df['Gender'] == 'Male') & (df['HasClaim'])]['ClaimSeverity'].dropna()
female_claims = df[(df['Gender'] == 'Female') & (df['HasClaim'])]['ClaimSeverity'].dropna()

t_stat, p_val = stats.ttest_ind(male_claims, female_claims, equal_var=False)
print(f"T-test for Claim Severity (Male vs Female): t = {t_stat:.2f}, p = {p_val:.4f}")


T-test for Claim Severity (Male vs Female): t = -0.58, p = 0.5680
